# Practice Using xlwings

Tutorial from:

https://www.dataquest.io/blog/python-excel-xlwings-tutorial/?utm_source=newsletter&utm_medium=email&utm_campaign=dataquest_newsletter

In [44]:
import pandas as pd
import xlwings as xw
df = pd.read_csv('euromillions.csv')
df.sample(5)

,No.,Day,DD,MMM,YYYY,N1,N2,N3,N4,N5,L1,L2,Jackpot,Wins
667,585,Tue,30,Apr,2013,13,50,40,43,36,9,5,12727500,0
923,329,Fri,28,May,2010,9,31,32,33,4,3,7,12693000,1
84,1168,Fri,30,Nov,2018,3,10,26,23,12,12,1,43374190,0
325,927,Tue,9,Aug,2016,19,43,44,12,45,10,5,25724734,0
819,433,Tue,15,Nov,2011,47,17,22,6,45,3,11,12817500,1


In [13]:
wb = xw.Book(r'C:\Users\jso\AppData\Local\Programs\Python\Python37\Scripts\EuroMillions.xlsx')

In [14]:
print(xw.apps)

Apps([<Excel App 5984>])


In [15]:
print(xw.apps[5984].books[0])

<Book [EuroMillions.xlsx]>


In [7]:
print(wb.sheets)

Sheets([<Sheet [EuroMillions.xlsx]Graphs>, <Sheet [EuroMillions.xlsx]Frequencies>, <Sheet [EuroMillions.xlsx]EuroMillions>])


In [16]:
print(xw.apps[5984].books[0] == wb)

True


Check to see the only instance of this iterable is the workbook wb.

In [39]:
print(wb.sheets[0])

<Sheet [EuroMillions.xlsx]123>


In [42]:
if not 'EuroMillions' in wb.names:
    ws = wb.sheets[0].name = "EuroMillions"
else:
    ws = wb.sheets["EuroMillions"]

In [9]:
wb.save("EuroMillions.xlsx")

In [46]:
ws.range("A1").value = df

In [49]:
ws.clear_contents()

In [53]:
ws.range("A1").options(index=False).value = df

Fills out worksheet with lottery info in df without the index.

In [ ]:
print(df.shape)

In [21]:
last_row = ws.range(1,1).end('down').row
print(f'The last row of the spreadsheet is {last_row}.')

The last row of the spreadsheet is 314.


In [15]:
ws.range(
    "A2:N{row}".format(row=last_row)
).api.Sort(Key1=ws.range("A:A").api, Order1=1)

True

In [16]:
ws.range("O1").value = "Date"
ws.range("O2").value = "=C2&D2&RIGHT(E2, 2)"

In [86]:
ws.range("O2").api.Autofill(ws.range(f"O2:O{last_row}").api,0)

True

In [11]:
from xlwings.constants import AutoFillType
ws.range("O2").api.AutoFill(
    ws.range(f"O2:O{last_row}").api,
    AutoFillType.xlFillDefault
)

True

Can use names of the fills by importing constants.

In [35]:
def autofill(worksheet, cell, last_row):
    rg_cell = worksheet.range(cell)
    col=rg_cell.get_address(0,0)[0]
    top_row=rg_cell.row
    last_row=last_row
    
    to_fill = f'{col}{top_row}:{col}{last_row}'
    rg_cell.api.Autofill(worksheet.range(to_fill).api, 0)

In [36]:
print(type(ws.range(f"O2:O{last_row}").value))

<class 'list'>


In [117]:
print(ws.range(f"O2:O{last_row}").value[:10])

[None, None, None, None, None, None, None, None, None, None]


In [98]:
ws.range('O2').options(transpose=True).value = ws.range(f"O2:O{last_row}").value

In [115]:
ws.range('C:E').api.Delete()

True

In [12]:
import datetime

for day in ws.range(f'L2:L{last_row}'):
    if day.value <= datetime.datetime(2016, 9, 24, 0, 0):
        to_delete = int(day.get_address(0,0)[1:])-1

ws.range(f'2:{to_delete}').api.Delete()

True

# Format Table

In [24]:
last_column = ws.range(1,1).end('right').get_address(0,0)[0]
ws.range(f'A1:{last_column}1').api.Font.Bold = True
ws.range(f'A1:{last_column}1').api.Borders(9).LineStyle = -4119
ws.range(f'J1:J{last_row}').number_format = '£#.###,, "M"'
ws.range("A:L").api.ColumnWidth = ws.range("J:J").api.ColumnWidth

# Frequency of Numbers

In [14]:
wb.sheets.add("Frequencies")
frequencies = wb.sheets["Frequencies"]

In [15]:
frequencies.range('A1').value = 'Number'
frequencies.range('A2:A51').value = '=ROW()-1'
frequencies.range('B1').value = 'Frequency'
frequencies.range('B2').value = '=COUNTIF(EuroMillions!$C$2:$I$314, Frequencies!A2)'
autofill(frequencies, 'B2', 51)

Count the number of numbers.

In [16]:
frequencies.range('D1').value = 'Lucky Stars'
frequencies.range('E1').value = 'Frequency'
frequencies.range('D2:D13').value = '=ROW()-1'
frequencies.range('E2:E13').value = '=COUNTIF(EuroMillions!$H$2:$I$314, Frequencies!D2)'

Count the number of Lucky Stars.

# Graphing the Numbers

In [17]:
wb.sheets.add("Graphs")

In [24]:
graphs = wb.sheets("Graphs")

#### Number Frequency Chart

In [18]:
nr_freq = xw.Chart()

In [37]:
nr_freq.name = 'Number Frequencies'
nr_freq.set_source_data(frequencies.range('Frequencies!B1:B51'))
nr_freq.api[1].FullSeriesCollection(1).XValues = 'Frequencies!A2:A51'
nr_freq.chart_type = 'column_clustered'
nr_freq.height = 250
nr_freq.width = 750
nr_freq.api[1].SetElement(2)
nr_freq.api[1].ChartTitle.Text = 'Number Frequencies'
nr_freq.api[1].HasLegend = 0
nr_freq.api[1].Axes(1).TickLabelSpacing = 1
graphs.shapes.api('Number Frequencies').Line.Visible = 0

#### Lucky Stars Chart

In [35]:
ls_freq = xw.Chart()

In [44]:
ls_freq.top = 250
ls_freq.name = 'Lucky Stars Frequencies'
ls_freq.set_source_data(frequencies.range('Frequencies!E2:E13'))
ls_freq.api[1].FullSeriesCollection(1).XValues = 'Frequencies!D2:D13'
ls_freq.chart_type = 'column_clustered'
ls_freq.height = 250
ls_freq.width = 750
ls_freq.api[1].SetElement(2)
ls_freq.api[1].ChartTitle.Text = 'Lucky Stars Frequencies'
ls_freq.api[1].HasLegend = 0
ls_freq.api[1].Axes(1).TickLabelSpacing = 1
graphs.shapes.api('Lucky Stars Frequencies').Line.Visible = 0

#### Jackpot Chart

In [22]:
jackpot = xw.Chart()

In [33]:
jackpot.top = 500
jackpot.name = 'Jackpot'
jackpot.set_source_data(ws.range(f'EuroMillions!J2:J{last_row}'))
jackpot.api[1].FullSeriesCollection(1).XValues = f'EuroMillions!L2:L{last_row}'
jackpot.chart_type = 'line'
jackpot.height = 250
jackpot.width = 750
jackpot.api[1].SetElement(2)
jackpot.api[1].ChartTitle.Text = 'Jackpot'
jackpot.api[1].HasLegend = 0
jackpot.api[1].Axes(1).TickLabelSpacing = 1
graphs.shapes.api('Jackpot').Line.Visible = 0
jackpot.api[1].Axes(2).TickLabels.NumberFormat = '£0.###,, "M"'

In [34]:
wb.save('EuroMillions.xlsx')